# Incorporating masks into calibrated science images

There are three ways of determining which pixels in a CCD image may need to be
masked (this is in addition to whatever mask or bit fields the observatory at
which you are taking images may provide).

Two of them are the same for all of the science images:

+ Hot pixels unlikely to be properly calibrated by subtracting dark current,
discussed in [Identifying hot pixels](08-01-Identifying-hot-pixels.ipynb).
+ Bad pixels identified by `ccdproc.ccdmask` from flat field images, discussed
in [Creating a mask with `ccdmask`](08-02-Creating-a-mask.ipynb).

The third, identifying cosmic rays, discussed in
[Cosmic ray removal](08-03-Cosmic-ray-removal.ipynb), will by its nature be different for each
science image.

The first two masks could be added to science images at the time the science
images are calibrated, if desired. They are added to the science images here, as
a separate step, because in many situations it is fine to omit masking entirely
and there is no particular advantage to introducing it earlier.

We begin, as usual, with a couple of imports.

In [ ]:
from pathlib import Path

from astropy import units as u
from astropy.nddata import CCDData

import ccdproc as ccdp

## Read masks that are the same for all of the science images

In previous notebooks we constructed a mask based on the dark current and a mask
created by `ccdmask` from a flat image. Displaying the summary of the the
information about the reduced images is a handy way to determine which files are
the masks.

In [ ]:
ex2_path = Path('example2-reduced')

ifc = ccdp.ImageFileCollection(ex2_path)
ifc.summary['file', 'imagetyp']

We read each of those in below, converting the mask to boolean after we read it.

In [ ]:
mask_ccdmask = CCDData.read(ex2_path / 'mask_from_ccdmask.fits', unit=u.dimensionless_unscaled)
mask_ccdmask.data = mask_ccdmask.data.astype('bool')

mask_hot_pix = CCDData.read(ex2_path / 'mask_from_dark_current.fits', unit=u.dimensionless_unscaled)
mask_hot_pix.data = mask_hot_pix.data.astype('bool')

### Combining the masks

We combine the masks using a logical "OR" since we want to mask out pixels that are
bad for any reason.

In [ ]:
combined_mask = mask_ccdmask.data | mask_hot_pix.data

It turns out we are masking roughly 0.056% of the pixels so far.

In [ ]:
combined_mask.sum()

## Detect cosmic rays

Cosmic ray detection was discussed in detail in an
[earlier section](08-03-Cosmic-ray-removal.ipynb). Here we loop over all of the calibrated
science images and:

+ detect cosmic rays in them,
+ combine the cosmic ray mask with the mask that applies to all images,
+ set the mask of the image to the overall mask, and
+ save the image, overwriting the calibrated science image without the mask.

Since the cosmic ray detection takes a while, a status message is displayed
before each image is processed.

In [ ]:
ifc.files_filtered()
for ccd, file_name in ifc.ccds(imagetyp='light', return_fname=True):
    print('Working on file {}'.format(file_name))
    new_ccd = ccdp.cosmicray_lacosmic(ccd, readnoise=10, sigclip=8, verbose=True)
    overall_mask = new_ccd.mask | combined_mask
    # If there was already a mask, keep it.
    if ccd.mask is not None:
        ccd.mask = ccd.mask | overall_mask
    else:
        ccd.mask = overall_mask
    # Files can be overwritten only with an explicit option
    ccd.write(ifc.location / file_name, overwrite=True)